<div align="center"><h1>Aplicação de técnicas de paralelismo para otimização de código</h1></div>

## Introdução

In [ ]:
%%writefile brute-force.c

// Sequential

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

// 97 to 122 use only lowercase letters
// 65 to 90 use only capital letters
// 48 to 57 use only numbers
#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

void bruteForce(char *pass);
long long my_pow(long long x, int y);

int main(int argc, char **argv) {
    char password[MAXIMUM_PASSWORD];
    strcpy(password, argv[1]);
    clock_t t1, t2, dif;

    t1 = clock();
    bruteForce(password);
    t2 = clock();

    dif = t2 - t1;

    double time_taken = ((double)dif)/CLOCKS_PER_SEC;

    printf("\n%1.2lf seconds\n", time_taken);

    return 0;
}

/*Check and increase the digits if you don't find the password...*/
void bruteForce(char *pass) {
    int size;
    int pass_b26[MAXIMUM_PASSWORD];
    long long int j;
    long long int pass_decimal = 0;
    int base = END_CHAR - START_CHAR + 2;

    size = strlen(pass);

    printf("Estamos tentando quebrar: %s\n", pass);

    for (int i = 0; i < size; i++) {
        pass_b26[i] = (int)pass[i] - START_CHAR + 1;  //+1 pois o vazio é o zero e o 'a' é 1
    }
    for (int i = size - 1; i > -1; i--) {
        pass_decimal += (long long int)pass_b26[i] * my_pow(base, i);
    }

    long long int max = my_pow(base, 9);
    char s[10];
    for (j = 0; j < max; j++) {
        if (j == pass_decimal) {
            printf("Encontrou o password!\n");
            int index = 0;

            printf("O número que estamos tentando encontrar (password na base decimal): %lli\n", j);
            while (j > 0) {
                s[index++] = START_CHAR + j % base - 1;
                j /= base;
            }
            s[index] = '\0';
            printf("Password encontrado: %s\n", s);
            break;
        }
    }
}

long long my_pow(long long x, int y) {
    long long res = 1;
    if (y == 0)
        return res;
    else
        return x * my_pow(x, y - 1);
}

Writing brute-force.c


In [ ]:
!gcc brute-force.c -o sequential; ./sequential zzzzzz

Estamos tentando quebrar: zzzzzz
Encontrou o password!
O número que estamos tentando encontrar (password na base decimal): 387420488
Password encontrado: zzzzzz

1.24 seconds


---
## Desenhos Paralelos

### Multicore - OPENMP

## Recursos 

In [ ]:
%%writefile openmp-info.cu

#include <stdio.h>
#include <openmp.h>

int main(){
  printf("numero de threads: %d\n", omp_get_num_threads);

  return 0;
}

Overwriting openmp-info.cu


In [ ]:
%%writefile brute-force-openmp.c

// OpenMP

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <omp.h>

// 97 to 122 use only lowercase letters
// 65 to 90 use only capital letters
// 48 to 57 use only numbers
#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

void bruteForce(char *pass);
long long my_pow(long long x, int y);
double t1, t2;
double dif;

int main(int argc, char **argv) {
    char password[MAXIMUM_PASSWORD];
    strcpy(password, argv[1]);
    t1 = omp_get_wtime();
    bruteForce(password);
    
    return 0;
}

/*Check and increase the digits if you don't find the password...*/
void bruteForce(char *pass) {
    int size;
    char force[MAXIMUM_PASSWORD];
    int palavra[9];
    int pass_b26[9];
    long long int j;
    long long int pass_decimal = 0;
    int base = END_CHAR - START_CHAR + 2;

    size = strlen(pass);

    for (int i = 0; i < MAXIMUM_PASSWORD; i++)
        force[i] = '\0';

    printf("Estamos tentando quebrar: %s\n", pass);

    for (int i = 0; i < strlen(pass); i++) {
        pass_b26[i] = (int)pass[i] - START_CHAR + 1;  //+1 pois o vazio é o zero e o 'a' é 1
    }

    for (int i = strlen(pass) - 1; i > -1; i--) {
        pass_decimal += (long long int)pass_b26[i] * my_pow(base, i);
    }

    long long int max = my_pow(base, 9);
    char s[10];
#pragma omp parallel for private(j)
    for (j = 0; j < max; j++) {
        if (j == pass_decimal) {
            printf("Encontrou o password!\n");
            int index = 0;

            printf("O número que estamos tentando encontrar (password na base decimal): %lli\n", j);
            while (j > 0) {
                s[index++] = START_CHAR + j % base - 1;
                j /= base;
            }
            s[index] = '\0';
            printf("Password encontrado: %s\n", s);
            t2 = omp_get_wtime();
            dif = t2 - t1;
            printf("\n%1.2lf seconds\n", dif);
            exit(1);
        }
    }
}

long long my_pow(long long x, int y) {
    long long res = 1;
    if (y == 0)
        return res;
    else
        return x * my_pow(x, y - 1);
}

Overwriting brute-force-openmp.c


In [ ]:
!gcc brute-force-openmp.c -o openmp -fopenmp; ./openmp zzzzzz 1

Estamos tentando quebrar: zzzzzz
Encontrou o password!
O número que estamos tentando encontrar (password na base decimal): 387420488
Password encontrado: zzzzzz

1.16 seconds


### Multiprocessador - MPI

In [ ]:
%%writefile mpi-info.c

#include <stdio.h>

int main(){
  // 

  return 0;
}

Writing mpi-info.c


In [ ]:
!mpicc mpi-info.c -o mpi-info; ./mpi-info

In [ ]:
%%writefile brute-force-mpi.c

// MPI

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <mpi.h>

// 97 to 122 use only lowercase letters
// 65 to 90 use only capital letters
// 48 to 57 use only numbers
#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

void bruteForce(char *pass);
long long my_pow(long long x, int y);
double t1, t2;
double dif;

int main(int argc, char **argv) {
    char password[MAXIMUM_PASSWORD];
    strcpy(password, argv[1]);

    MPI_Init(&argc, &argv);

    t1 = MPI_Wtime();
    bruteForce(password);

    return 0;
}

/*Check and increase the digits if you don't find the password...*/
void bruteForce(char *pass) {
    int size;
    int pass_b26[MAXIMUM_PASSWORD];
    long long int j;
    long long int pass_decimal = 0;
    int base = END_CHAR - START_CHAR + 2;

    size = strlen(pass);

    printf("Estamos tentando quebrar: %s\n", pass);

    for (int i = 0; i < size; i++) {
        pass_b26[i] = (int)pass[i] - START_CHAR + 1;  //+1 pois o vazio é o zero e o 'a' é 1
    }

    for (int i = size - 1; i > -1; i--) {
        pass_decimal += (long long int)pass_b26[i] * my_pow(base, i);
    }

    long long int max = my_pow(base, 9);
    char s[10];

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int numberOfProcesses;
    MPI_Comm_size(MPI_COMM_WORLD, &numberOfProcesses);

    long long int amountPerProcess = max / numberOfProcesses;

    // 4 processos
    // 0  1  2  3
    // c  c  c  c

    // 0: amountPerProcess*0 ate amountPerProcess*1
    // 1: amountPerProcess*1 ate amountPerProcess*2
    // 2: amountPerProcess*2 ate amountPerProcess*3
    // 3: amountPerProcess*3 ate amountPerProcess*4

    for (int n = 0; n < numberOfProcesses; n++) {
        if (rank == n) {
            for (j = amountPerProcess * n; j < amountPerProcess * (n + 1); j++) {
                if (j == pass_decimal) {
                    printf("Encontrou o password!\n");
                    int index = 0;

                    printf("O número que estamos tentando encontrar (password na base decimal): %lli\n", j);
                    while (j > 0) {
                        s[index++] = START_CHAR + j % base - 1;
                        j /= base;
                    }
                    s[index] = '\0';
                    printf("Password encontrado: %s\n", s);
                    t2 = MPI_Wtime();

                    dif = t2 - t1;

                    printf("\n%1.2f seconds\n", dif);
                    MPI_Abort(MPI_COMM_WORLD, 1);
                    MPI_Finalize();
                }
            }
        }
    }
}

long long my_pow(long long x, int y) {
    long long res = 1;
    if (y == 0)
        return res;
    else
        return x * my_pow(x, y - 1);
}

Writing brute-force-mpi.c


In [ ]:
!mpicc brute-force-mpi.c -o mpi; mpirun -np 1 --allow-run-as-root --oversubscribe ./mpi zzzzzz

Estamos tentando quebrar: zzzzzz
Encontrou o password!
O número que estamos tentando encontrar (password na base decimal): 387420488
Password encontrado: zzzzzz

1.05 seconds
--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD
with errorcode 1.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------


### Híbrido (OpenMP + MPI)

In [ ]:
%%writefile brute-force-hibrido.c

// OpenMP + MPI

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <mpi.h>
#include <omp.h>


// 97 to 122 use only lowercase letters
// 65 to 90 use only capital letters
// 48 to 57 use only numbers
#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

void bruteForce(char *pass);
long long my_pow(long long x, int y);
double t1, t2;
double dif;

int main(int argc, char **argv) {
    char password[MAXIMUM_PASSWORD];
    strcpy(password, argv[1]);

    MPI_Init(&argc, &argv);

    t1 = MPI_Wtime();
    bruteForce(password);

    return 0;
}

/*Check and increase the digits if you don't find the password...*/
void bruteForce(char *pass) {
    int size;
    int pass_b26[MAXIMUM_PASSWORD];
    long long int j;
    long long int pass_decimal = 0;
    int base = END_CHAR - START_CHAR + 2;

    size = strlen(pass);

    printf("Estamos tentando quebrar: %s\n", pass);

    for (int i = 0; i < size; i++) {
        pass_b26[i] = (int)pass[i] - START_CHAR + 1;  //+1 pois o vazio é o zero e o 'a' é 1
    }

    for (int i = size - 1; i > -1; i--) {
        pass_decimal += (long long int)pass_b26[i] * my_pow(base, i);
    }

    long long int max = my_pow(base, 9);
    char s[10];

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int numberOfProcesses;
    MPI_Comm_size(MPI_COMM_WORLD, &numberOfProcesses);

    long long int amountPerProcess = max / numberOfProcesses;

    // 4 processos
    // 0  1  2  3
    // c  c  c  c

    // 0: amountPerProcess*0 ate amountPerProcess*1
    // 1: amountPerProcess*1 ate amountPerProcess*2
    // 2: amountPerProcess*2 ate amountPerProcess*3
    // 3: amountPerProcess*3 ate amountPerProcess*4

    for (int n = 0; n < numberOfProcesses; n++) {
        if (rank == n) {
            #pragma omp parallel for private(j)
            for (j = amountPerProcess * n; j < amountPerProcess * (n + 1); j++) {
                if (j == pass_decimal) {
                    printf("Encontrou o password!\n");
                    int index = 0;

                    printf("O número que estamos tentando encontrar (password na base decimal): %lli\n", j);
                    while (j > 0) {
                        s[index++] = START_CHAR + j % base - 1;
                        j /= base;
                    }
                    s[index] = '\0';
                    printf("Password encontrado: %s\n", s);
                    t2 = MPI_Wtime();

                    dif = t2 - t1;

                    printf("\n%1.2f seconds\n", dif);
                    MPI_Abort(MPI_COMM_WORLD, 1);
                    MPI_Finalize();
                    exit(1);
                }
            }
        }
    }
}

long long my_pow(long long x, int y) {
    long long res = 1;
    if (y == 0)
        return res;
    else
        return x * my_pow(x, y - 1);
}

Writing brute-force-hibrido.c


In [ ]:
!mpicc brute-force-hibrido.c -o hibrido -fopenmp; OMP_NUM_THREADS=2 mpirun -np 1 --allow-run-as-root --oversubscribe ./hibrido zzzzzz

Estamos tentando quebrar: zzzzzz
Encontrou o password!
O número que estamos tentando encontrar (password na base decimal): 387420488
Password encontrado: zzzzzz

0.96 seconds
--------------------------------------------------------------------------
MPI_ABORT was invoked on rank 0 in communicator MPI_COMM_WORLD
with errorcode 1.

NOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.
You may or may not see output from other processes, depending on
exactly when Open MPI kills them.
--------------------------------------------------------------------------


### GPU - CUDA

## Recursos da GPU

In [ ]:
%%writefile cuda-info.cu

#include <stdio.h>
#include <cuda.h>

int main(){
  cudaDeviceProp deviceProp;
  cudaGetDeviceProperties(&deviceProp, 0);  // 0-th device
  printf("numero de multiprocessadores: %d\n", deviceProp.multiProcessorCount);
  printf("numero de threads por bloco: %d\n", deviceProp.maxThreadsPerBlock);

  return 0;
}

Writing cuda-info.cu


In [ ]:
!nvcc cuda-info.cu -o cuda-info; ./cuda-info

numero de multiprocessadores: 40
numero de threads por bloco: 1024


In [ ]:
%%writefile brute-force.cu

// CUDA

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>
#include <cuda.h>
#include <unistd.h>

// 97 to 122 use only lowercase letters
// 65 to 90 use only capital letters
// 48 to 57 use only numbers
#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

__global__ void bruteForce(char *pass, unsigned int size);
__host__ __device__ long long my_pow(long long x, int y);
__host__ __device__ unsigned int my_strlen(char *s);

int main(int argc, char **argv) {
    size_t size = sizeof(char) * MAXIMUM_PASSWORD;
    char *password;
    cudaMallocManaged(&password, size);
    strcpy(password, argv[1]);

    size_t threadsPerBlock = 1024;
    size_t numberOfBlocks = 40;

    printf("Estamos tentando quebrar: %s\n", password);

    unsigned int numberOfCharacters = my_strlen(password);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    bruteForce<<<numberOfBlocks, threadsPerBlock>>>(password, numberOfCharacters);
    cudaEventRecord(stop);
    
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    printf("\n%1.2f seconds\n", milliseconds/1000);

    return 0;
}

/*Check and increase the digits if you don't find the password...*/
__global__ void bruteForce(char *pass, unsigned int size) {
    int pass_b26[MAXIMUM_PASSWORD];
    long long int j;
    long long int pass_decimal = 0;
    int base = END_CHAR - START_CHAR + 2;

    for (int i = 0; i < size; i++) {
        pass_b26[i] = (int)pass[i] - START_CHAR + 1;  //+1 pois o vazio é o zero e o 'a' é 1
    }

    for (int i = size - 1; i > -1; i--) {
        pass_decimal += (long long int)pass_b26[i] * my_pow(base, i);
    }

    long long int max = my_pow(base, 9);
    char s[10];
    for (j = blockIdx.x * blockDim.x + threadIdx.x; j < max; j += blockDim.x * gridDim.x) {
        if (j == pass_decimal) {
            printf("Encontrou o password!\n");
            int index = 0;

            printf("O número que estamos tentando encontrar (password na base decimal): %lli\n", j);
            while (j > 0) {
                s[index++] = START_CHAR + j % base - 1;
                j /= base;
            }
            s[index] = '\0';
            printf("Password encontrado: %s\n", s);
            break;
        } else if (j > pass_decimal){
          break;
        }
    }
}

__host__ __device__ long long my_pow(long long x, int y) {
    long long res = 1;
    if (y == 0)
        return res;
    else
        return x * my_pow(x, y - 1);
}

unsigned int my_strlen(char *palavra) {
    int i = 0;

    while (palavra[i] != '\0') {
        i++;
    }

    return i;
}

Overwriting brute-force.cu


In [ ]:
!nvcc brute-force.cu -o cuda; ./cuda zzzzzz

Estamos tentando quebrar: zzzzzz
Encontrou o password!
O número que estamos tentando encontrar (password na base decimal): 387420488
Password encontrado: zzzzzz

0.00 seconds


## Modelo híbrido

## Análise Experimental

1. OpenMP 
2. MPI 
3. MPI + OpenMP 
4. CUDA

### Tempo de execução em segundas das aplicações

|  Senha    | Sequencial | OpenMP | MPI  | Híbrido | CUDA
| --------- | ---------- | ------ | ---  | ------- | ----
|         |           |       |     |       |  
|         |         |      |    |        | 
|         |        |      |   |        | 
|        |       |    | |      | 

### Speedup

|  Senha    |  OpenMP    | MPI     | Híbrido       | CUDA
| --------- |  ------    | ------  | -------       | ----
|         |        |      |            |  
|         |       |    |            |   
|        |       |     |         |  
|      |       |    |         |  